In [1]:
import qctoolkit as qtk
import numpy as np
import sys

/home/samio/.local/miniconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
E_dict = {
    'H': [ -0.500273,  -0.498857,  -0.497912,  -0.510927],
    'C': [-37.846772, -37.845355, -37.844411, -37.861317],
    'N': [-54.583861, -54.582445, -54.581501, -54.598897],
    'O': [-75.064579, -75.063163, -75.062219, -75.079532],
    'F': [-99.718730, -99.717314, -99.716370, -99.733544],
}

def getEa(mol, properties):
    atomized_energies = np.array(properties[10:14])
    mol_count = mol.stoichiometry(output='dictionary')
    for atom, count in mol_count.items():
        atomized_energies -= count * np.array(E_dict[atom])
    return atomized_energies * qtk.convE(1, "Eh-kcal")[0]

In [3]:
# load data

mols = []
properties = []
atomized = []
smiles_original = []
smiles_optimized = []
for i in range(1, 133886):
#for i in range(1, 1000):
    name = 'qm9/dsgdb9nsd_%06d.xyz' % i
    mol = qtk.Molecule(name)
    mol_file = open(name)
    mol_data = mol_file.readlines()
    mol_prop = [float(p) for p in mol_data[1].split("\t")[1:-1]]
    properties.append(mol_prop)
    atomized.append(getEa(mol, mol_prop))
    smiles_original.append(mol_data[-2].split("\t")[0])
    smiles_optimized.append(mol_data[-2].split("\t")[1])
    mols.append(mol)
    mol_file.close()
    if i % 10000 == 1:
        print "processing %d" % i
properties = np.array(properties)
Ea_prop = np.stack(atomized)

processing 1
processing 10001
processing 20001
processing 30001
processing 40001
processing 50001
processing 60001
processing 70001
processing 80001
processing 90001
processing 100001
processing 110001
processing 120001
processing 130001


In [4]:
# construct numpy data
data = qtk.ML.pack(mols)

processing 1
processing 5001
processing 10001
processing 15001
processing 20001
processing 25001
processing 30001
processing 35001
processing 40001
processing 45001
processing 50001
processing 55001
processing 60001
processing 65001
processing 70001
processing 75001
processing 80001
processing 85001
processing 90001
processing 95001
processing 100001
processing 105001
processing 110001
processing 115001
processing 120001
processing 125001
processing 130001


In [5]:
data['E'] = Ea_prop[:,1]
data['atomized_energies'] = Ea_prop
data['properties'] = properties
data['smiles_original'] = smiles_original
data['smiles_optimized'] = smiles_optimized

In [6]:
qtk.pdump(data, "data_qm9.pkl")